In [2]:
import matplotlib.pyplot as plt
from pprint import pprint as pp
import numpy as np

feature_matrix = []
target_vector1 = []
target_vector2 = []
varToNumNA = dict()

for line in open('CommViolPredUnnormalizedData.txt', 'r'):
    features_orig = line.strip().split(',')
    for i in range(len(features_orig)):
        if features_orig[i] == '?':
            try:
                varToNumNA[i] += 1
            except:
                varToNumNA[i] = 1
    
    target1 = features_orig[-2] # ViolentCrimesPerPop
    target2 = features_orig[-1] # nonViolPerPop
    #features = [ f for f in features[3:-2]] # don't include town and state name
    features = [ f for f in features_orig[4:103] ] #don't include 103
    features += [f for f in features_orig[120:123]] #don't include 103-119, or 123
    features += [features_orig[127]] 
    features += [f for f in features_orig[129:131]]
    features += [f for f in features_orig[133:143]]
    feature_matrix.append(features)
    target_vector1.append(target1)
    target_vector2.append(target2)
  

In [3]:
# don't use the variables that have a lot of '?'s in th data
varToNumNA # {var : numNA}, var is the index of the variable, numNA is the nubmer of ?s

{2: 1221,
 3: 1224,
 30: 1,
 103: 1872,
 104: 1872,
 105: 1872,
 106: 1872,
 107: 1872,
 108: 1872,
 109: 1872,
 110: 1872,
 111: 1872,
 112: 1872,
 113: 1872,
 114: 1872,
 115: 1872,
 116: 1872,
 117: 1872,
 118: 1872,
 119: 1872,
 123: 1872,
 124: 1872,
 125: 1872,
 126: 1872,
 128: 1872,
 131: 208,
 132: 208,
 133: 1,
 134: 1,
 135: 13,
 136: 13,
 137: 3,
 138: 3,
 139: 3,
 140: 3,
 141: 3,
 142: 3,
 143: 91,
 144: 91,
 145: 221,
 146: 97}

In [4]:
len(features)

115

In [5]:
feature_matrix[1]
'?' in feature_matrix[1]

False

In [6]:
feature_matrix_clean = []
target_vector1_clean = []
target_vector2_clean = []
for i in range(len(feature_matrix)):
    if ('?' not in feature_matrix[i] and '?' not in target_vector1[i] and '?' not in target_vector2[i]):
        feature_matrix_clean.append([float(x) for x in feature_matrix[i]])
        target_vector1_clean.append(float(target_vector1[i]))
        target_vector2_clean.append(float(target_vector2[i]))

In [7]:
len(feature_matrix), len(feature_matrix_clean) # get rid of a lot data, maybe remove variables instead?

(2215, 1901)

In [8]:
data = np.array( feature_matrix_clean )
target1 = np.array( [ (1 if (x > 600) else 0) for x in target_vector1_clean] )
target2 = np.array( [ (1 if (x > 600) else 0) for x in target_vector2_clean] )

print(data)
print(target1)

[[  1.00000000e+00   1.19800000e+04   3.10000000e+00 ...,   1.13208000e+03
    1.60000000e+01   1.31260000e+02]
 [  1.00000000e+00   2.31230000e+04   2.82000000e+00 ...,   1.59878000e+03
    2.60000000e+01   1.10550000e+02]
 [  1.00000000e+00   2.93440000e+04   2.43000000e+00 ...,   4.97219000e+03
    1.36000000e+02   3.76300000e+02]
 ..., 
 [  1.00000000e+01   3.28240000e+04   2.46000000e+00 ...,   2.43597000e+03
    1.79000000e+02   4.87190000e+02]
 [  1.00000000e+01   1.35470000e+04   2.89000000e+00 ...,   3.72290000e+03
    1.30000000e+01   1.02100000e+02]
 [  1.00000000e+01   2.88980000e+04   2.61000000e+00 ...,   4.81920000e+03
    4.05000000e+02   1.33867000e+03]]
[0 0 0 ..., 0 1 1]


In [9]:
# We will use a variation of NB 
from sklearn.naive_bayes import GaussianNB

# fit a Naive Bayes model to the data
model = GaussianNB()
X_train, y_train1 = data, target1 
model.fit(X_train, y_train1)

GaussianNB(priors=None)

In [10]:
y_predicted = model.predict(X_train) 
y_expected = y_train1

In [11]:
# Import  metrics
from sklearn import metrics

# summarize the fit of the model

print(metrics.accuracy_score(y_expected, y_predicted))
print()
print(metrics.classification_report(y_expected, y_predicted))
print(metrics.confusion_matrix(y_expected, y_predicted))
print()

0.862703840084

             precision    recall  f1-score   support

          0       0.85      0.97      0.90      1269
          1       0.91      0.65      0.76       632

avg / total       0.87      0.86      0.86      1901

[[1229   40]
 [ 221  411]]



In [12]:
def readFile(filename, mode="rt"):
    # rt stands for "read text"
    fin = contents = None
    try:
        fin = open(filename, mode)
        contents = fin.read()
    finally:
        if (fin != None): fin.close()
    return contents

#def indexToName(i):
#    contents = readFile('varNames.txt')
#    contents_list = contents.split('\n')
#    contents_list = [ (s.split())[1][:-1] for s in contents_list ]
#    return contents_list[i]

contents = readFile('varNames.txt')
contents_list = contents.split('\n')
contents_list = [ (s.split())[1][:-1] for s in contents_list ]
#contents_list.index('population')

In [13]:
varNames = []
for i in varToNumNA:
    varNames += [(i, varToNumNA[i], contents_list[i])]
sorted(varNames) # variables that we didn't use

[(2, 1221, 'countyCode'),
 (3, 1224, 'communityCode'),
 (30, 1, 'OtherPerCap'),
 (103, 1872, 'LemasSwornFT'),
 (104, 1872, 'LemasSwFTPerPop'),
 (105, 1872, 'LemasSwFTFieldOps'),
 (106, 1872, 'LemasSwFTFieldPerPop'),
 (107, 1872, 'LemasTotalReq'),
 (108, 1872, 'LemasTotReqPerPop'),
 (109, 1872, 'PolicReqPerOffic'),
 (110, 1872, 'PolicPerPop'),
 (111, 1872, 'RacialMatchCommPol'),
 (112, 1872, 'PctPolicWhite'),
 (113, 1872, 'PctPolicBlack'),
 (114, 1872, 'PctPolicHisp'),
 (115, 1872, 'PctPolicAsian'),
 (116, 1872, 'PctPolicMinor'),
 (117, 1872, 'OfficAssgnDrugUnits'),
 (118, 1872, 'NumKindsDrugsSeiz'),
 (119, 1872, 'PolicAveOTWorked'),
 (123, 1872, 'PolicCars'),
 (124, 1872, 'PolicOperBudg'),
 (125, 1872, 'LemasPctPolicOnPatr'),
 (126, 1872, 'LemasGangUnitDeploy'),
 (128, 1872, 'PolicBudgPerPop'),
 (131, 208, 'rapes'),
 (132, 208, 'rapesPerPop'),
 (133, 1, 'robberies'),
 (134, 1, 'robbbPerPop'),
 (135, 13, 'assaults'),
 (136, 13, 'assaultPerPop'),
 (137, 3, 'burglaries'),
 (138, 3, 'burgl

In [14]:
from sklearn import linear_model, datasets
logreg = linear_model.LogisticRegression(C=1e5, penalty='l1',)
X = feature_matrix_clean
y = [ (1 if (x > 600) else 0) for x in target_vector1_clean]
logreg.fit(X, y)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [15]:
y_predicted_log = logreg.predict(X)
y_expected = y_train1
print(metrics.accuracy_score(y_expected, y_predicted_log))
print()
print(metrics.classification_report(y_expected, y_predicted_log))
print(metrics.confusion_matrix(y_expected, y_predicted_log))
print()

0.99158337717

             precision    recall  f1-score   support

          0       0.99      1.00      0.99      1269
          1       0.99      0.98      0.99       632

avg / total       0.99      0.99      0.99      1901

[[1264    5]
 [  11  621]]

